<a href="https://colab.research.google.com/github/Gihan007/Corrective_RAG_systems-/blob/main/Raptor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain langchain-community langchain-openai faiss-cpu pypdf tiktoken openai python-dotenv matplotlib


INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32

In [65]:

import numpy as np
import pandas as pd
from typing import List, Dict, Any
import os
from dotenv import load_dotenv

# NirDiamant's key imports - these are essential for his methodology
from sklearn.mixture import GaussianMixture  # For clustering (better than K-means)
from langchain.chains.llm import LLMChain
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.schema import AIMessage, Document
from langchain_community.document_loaders import PyPDFLoader

import matplotlib.pyplot as plt
print("All packages imported successfully!")

All packages imported successfully!


In [67]:
import re
from langchain_community.document_loaders import PyPDFLoader
import numpy as np

def calculate_mean_sentence_length(document_path: str) -> float:
    """
    Loads text from a PDF, splits it into sentences, calculates the word count
    for each sentence, and returns the mean average length.
    """
    try:
        # 1. Load the PDF and extract text from all pages
        loader = PyPDFLoader(document_path)
        documents = loader.load()

        # Combine all page content into a single large string
        full_text = " ".join([doc.page_content for doc in documents])

    except FileNotFoundError:
        print(f"ERROR: File not found at '{document_path}'. Please check the path.")
        return 0.0
    except Exception as e:
        print(f"An error occurred during PDF loading: {e}")
        return 0.0

    # 2. Sentence Tokenization
    # Use regular expression to split the text into sentences
    # (splitting on '.', '!', or '?' followed by a space or end-of-string)
    sentences = re.split(r'[.!?]\s*', full_text)

    # Filter out empty strings that might result from splitting (e.g., at the end of the text)
    sentences = [s.strip() for s in sentences if s.strip()]

    if not sentences:
        print("No sentences found in the document.")
        return 0.0

    # 3. Word Tokenization and Length Calculation
    sentence_lengths = []

    for sentence in sentences:
        # A simple split by space is sufficient to count words.
        # Filter out empty strings that result from multiple spaces.
        words = [word for word in sentence.split() if word]
        sentence_lengths.append(len(words))

    # 4. Calculate the Mean Average Length
    mean_length = np.mean(sentence_lengths)

    print(f"\n--- Analysis Complete ---")
    print(f"Total Sentences Analyzed: {len(sentence_lengths)}")
    print(f"Average Sentence Length (in words): {mean_length:.2f}")

    return mean_length

# --- EXECUTION ---
# IMPORTANT: Replace "hey.pdf" with the actual path to your document.
pdf_file = "hey.pdf"

mean_avg_length = calculate_mean_sentence_length(pdf_file)

if mean_avg_length > 0:
    print(f"\nThe mean average sentence length is {mean_avg_length:.2f} words.")


--- Analysis Complete ---
Total Sentences Analyzed: 415
Average Sentence Length (in words): 9.77

The mean average sentence length is 9.77 words.


In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import os # Ensure os is imported for path handling

# --- 1. Load the Document Pages ---
document_path = "hey.pdf"
loader = PyPDFLoader(document_path)
documents = loader.load()

# 'texts' is now a list of strings, where each string is a page's full content.
texts = [doc.page_content for doc in documents]
page_count = len(documents)

print(f"Loaded {page_count} pages from the AI/ML document.")
print("-" * 50)

# --- 2. Split Pages into Granular Chunks (The essential step for RAG/RAPTOR) ---

# Initialize the Text Splitter
# These parameters are standard for creating good, overlapping chunks.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,     # The size of each chunk
    chunk_overlap=5   # Overlap to preserve context across chunks
)

# Convert the list of raw page texts into a list of LangChain Document objects
# before splitting, so metadata (like page number) is carried forward.
documents_for_splitting = [
    Document(page_content=text, metadata={"source_page": i})
    for i, text in enumerate(texts)
]

# Perform the actual splitting
# 'all_chunks' now contains the Level 0 leaf nodes for RAPTOR.
all_chunks = text_splitter.split_documents(documents_for_splitting)

print(f"Divided {page_count} pages into {len(all_chunks)} granular chunks (Level 0 leaf nodes).")
print(f"First chunk preview: {all_chunks[0].page_content[:400]}...")
  # Display metadata of the first chunk

# NOTE: You should now use the 'all_chunks' list, not 'texts',
# as the input for your create_embeddings_batch function.

Loaded 6 pages from the AI/ML document.
--------------------------------------------------
Divided 6 pages into 239 granular chunks (Level 0 leaf nodes).
First chunk preview: 1 
 
Fixed Income Mutual Fund Performance: Pre, During, and Post- 
COVID-19 and Sovereign Debt Crisis in Sri Lanka...


In [68]:
print(all_chunks)

[Document(metadata={'source_page': 0}, page_content='1 \n \nFixed Income Mutual Fund Performance: Pre, During, and Post- \nCOVID-19 and Sovereign Debt Crisis in Sri Lanka'), Document(metadata={'source_page': 0}, page_content='J.A. V Sithumini (*)1, H.M.N.P. Herath2 \nDepartment of Finance, University of Kelaniya, Sri Lanka1,2 \nvinodyajvs@gmail.com1, nethmih@kln.ac.lk2'), Document(metadata={'source_page': 0}, page_content='Abstract  \nThis study empirically assessed the performance of fixed -income mutual funds highlighting the pre, during, and post'), Document(metadata={'source_page': 0}, page_content='COVID-19 and Sovereign Debt Crisis  periods of Sri Lanka to fill the gap of the lack of  focus on the fixed -income'), Document(metadata={'source_page': 0}, page_content='mutual fund performance in emerging markets.  This study collected data covering the entire Sri Lankan fixed -income'), Document(metadata={'source_page': 0}, page_content='mutual fund market from Unit Trust Association

In [73]:
texts = []  # create an empty list

for chunk in all_chunks:
    texts.append(chunk.page_content)

print(f"Stored {len(texts)} chunks of text.")
print(texts[2])  # preview first 300 characters


Stored 239 chunks of text.
Abstract  
This study empirically assessed the performance of fixed -income mutual funds highlighting the pre, during, and post


In [74]:
# Initialize AI components
embeddings = OpenAIEmbeddings(api_key="sk-proj-")
llm= ChatOpenAI(model_name="gpt-4o-mini" , api_key="sk-proj-")

def create_embeddings_batch(texts_list, embeddings_model):
    """Convert texts to embeddings - unified function to avoid duplication."""
    print(f" Creating embeddings for {len(texts_list)} texts...")
    all_embeddings = []

    for text in texts_list:
        # Handle different text formats
        text_content = text.content if hasattr(text, 'content') else str(text)
        embedding = embeddings_model.embed_documents([text_content])
        all_embeddings.extend(embedding)  # This adds the embedding vector to the list

    print(f" Created {len(all_embeddings)} embeddings")
    return all_embeddings


In [75]:
def cluster_with_gmm(embeddings_array, n_clusters):
    """Cluster embeddings using Gaussian Mixture Models."""
    # Ensure we don't try to create more clusters than data points
    n_clusters = min(n_clusters, len(embeddings_array))
    if n_clusters < 2:
        return np.zeros(len(embeddings_array))  # Single cluster

    print(f" Clustering into {n_clusters} groups...")
    gm = GaussianMixture(n_components=n_clusters, random_state=42)
    return gm.fit_predict(embeddings_array)

In [77]:
def generate_summaries_for_clusters(texts, cluster_labels, level_number, llm_model):
    """Generate summaries for each cluster."""
    print(f" Generating summaries for Level {level_number}...")
    summaries = []
    metadata_list = []

    unique_clusters = np.unique(cluster_labels)
    for cluster_id in unique_clusters:
        # Get texts in this cluster
        cluster_mask = cluster_labels == cluster_id
        cluster_texts = [texts[i] for i in range(len(texts)) if cluster_mask[i]]

        # Combine and summarize
        combined_text = "\n\n".join(cluster_texts)
        prompt = ChatPromptTemplate.from_template(
            "Summarize the following content concisely, focusing on key concepts and relationships:\n\n{text}"
        )
        chain = prompt | llm_model
        result = chain.invoke({"text": combined_text[:4000]})  # Limit context

        summary = result.content if hasattr(result, 'content') else str(result)
        summaries.append(summary)

        # Create metadata
        metadata = {
            "level": level_number,
            "cluster_id": int(cluster_id),
            "source_docs": len(cluster_texts),
            "id": f"summary_L{level_number}_C{cluster_id}"
        }
        metadata_list.append(metadata)

    return summaries, metadata_list

In [78]:
def create_dataframe_for_level(texts, embeddings_list, metadata_list, cluster_labels=None):
    """Create organized DataFrame for each tree level."""
    df = pd.DataFrame({
        'text': texts,
        'embedding': embeddings_list,
        'metadata': metadata_list
    })
    if cluster_labels is not None:
        df['cluster'] = cluster_labels
    return df


In [79]:
# Build Level 0 - Original documents
print(" Building Level 0 (Original Documents)...")

# Create embeddings for all original texts
embeddings_list = create_embeddings_batch(texts, embeddings)

# Create metadata for original documents
level_0_metadata = []
for i, text in enumerate(texts):
    metadata = {
        "level": 0,
        "origin": "original",
        "id": f"doc_{i}",
        "text_length": len(text)
    }
    level_0_metadata.append(metadata)

# Create Level 0 DataFrame
level_0_df = create_dataframe_for_level(
    texts,
    embeddings_list,  # Use the list directly
    level_0_metadata
)

print(f" Level 0 created with {len(level_0_df)} documents")
print(f" Sample metadata: {level_0_df['metadata'].iloc[0]}")

# Store in our tree structure
tree_levels = {0: level_0_df}

# Convert to numpy array for clustering (we need numpy array for GMM)
embeddings_array = np.array(embeddings_list)

 Building Level 0 (Original Documents)...
 Creating embeddings for 239 texts...
 Created 239 embeddings
 Level 0 created with 239 documents
 Sample metadata: {'level': 0, 'origin': 'original', 'id': 'doc_0', 'text_length': 114}


In [80]:
# Cluster Level 0 documents
optimal_clusters = min(8, max(2, len(texts) // 3))  # Smart cluster sizing
cluster_labels = cluster_with_gmm(embeddings_array, optimal_clusters)

# Add clusters to Level 0
level_0_df['cluster'] = cluster_labels

print(" Clustering Results:")
for cluster_id in np.unique(cluster_labels):
    count = sum(cluster_labels == cluster_id)
    print(f"  Cluster {cluster_id}: {count} documents")

# Generate Level 1 summaries
level_1_summaries, level_1_metadata = generate_summaries_for_clusters(
    texts, cluster_labels, level_number=1, llm_model=llm
)

# Create embeddings for summaries
level_1_embeddings = create_embeddings_batch(level_1_summaries, embeddings)

# Create Level 1 DataFrame
level_1_df = create_dataframe_for_level(
    level_1_summaries,
    level_1_embeddings,  # Remove list() conversion since it's already a list
    level_1_metadata
)

# Add to tree
tree_levels[1] = level_1_df

print(f" Level 1 created with {len(level_1_df)} summaries")
print(f" Sample summary: {level_1_summaries[0][:150]}...")


 Clustering into 8 groups...
 Clustering Results:
  Cluster 0: 14 documents
  Cluster 1: 13 documents
  Cluster 2: 34 documents
  Cluster 3: 31 documents
  Cluster 4: 48 documents
  Cluster 5: 35 documents
  Cluster 6: 28 documents
  Cluster 7: 36 documents
 Generating summaries for Level 1...
 Creating embeddings for 8 texts...
 Created 8 embeddings
 Level 1 created with 8 summaries
 Sample summary: The provided content discusses the relationship between short-term (3-Month Treasury Bill yield) and long-term (10-Year Government Bond yield) financi...


In [81]:
# Continue building levels until we have few enough summaries
current_level = 1
current_texts = level_1_summaries
current_embeddings = level_1_embeddings

# Convert to numpy array for clustering
current_embeddings_array = np.array(current_embeddings)

while len(current_texts) > 3 and current_level < 4:  # Prevent infinite loops
    print(f"\n Building Level {current_level + 1}...")

    # Cluster current level
    n_clusters = min(6, max(2, len(current_texts) // 2))
    cluster_labels = cluster_with_gmm(current_embeddings_array, n_clusters)

    # Add clusters to current level DataFrame (this is safe now)
    if current_level in tree_levels:
        tree_levels[current_level]['cluster'] = cluster_labels
        print(f" Added cluster info to Level {current_level}")

    # Generate summaries for next level
    next_level = current_level + 1
    next_summaries, next_metadata = generate_summaries_for_clusters(
        current_texts, cluster_labels, next_level, llm
    )

    # Check if we got any summaries
    if not next_summaries:
        print(f" No summaries generated for Level {next_level}, stopping.")
        break

    # Create embeddings for next level
    next_embeddings = create_embeddings_batch(next_summaries, embeddings)

    # Create next level DataFrame
    next_df = create_dataframe_for_level(
        next_summaries,
        next_embeddings,
        next_metadata
    )

    # Add to tree
    tree_levels[next_level] = next_df

    print(f" Level {next_level} created with {len(next_df)} summaries")

    # Prepare for next iteration
    current_level = next_level
    current_texts = next_summaries
    current_embeddings = next_embeddings
    current_embeddings_array = np.array(current_embeddings)

print(f"\n Final tree structure:")
for level, df in tree_levels.items():
    content_type = "original documents" if level == 0 else f"level-{level} summaries"
    print(f"  Level {level}: {len(df)} {content_type}")


 Building Level 2...
 Clustering into 4 groups...
 Added cluster info to Level 1
 Generating summaries for Level 2...
 Creating embeddings for 4 texts...
 Created 4 embeddings
 Level 2 created with 4 summaries

 Building Level 3...
 Clustering into 2 groups...
 Added cluster info to Level 2
 Generating summaries for Level 3...
 Creating embeddings for 2 texts...
 Created 2 embeddings
 Level 3 created with 2 summaries

 Final tree structure:
  Level 0: 239 original documents
  Level 1: 8 level-1 summaries
  Level 2: 4 level-2 summaries
  Level 3: 2 level-3 summaries


In [82]:
# Build unified FAISS vectorstore from all tree levels
print(" Building unified FAISS vectorstore...")

all_documents = []
level_distribution = {}

# Collect documents from all levels
for level, df in tree_levels.items():
    print(f" Adding Level {level} to vectorstore...")

    for _, row in df.iterrows():
        doc = Document(
            page_content=str(row['text']),
            metadata=row['metadata']
        )
        all_documents.append(doc)

    level_distribution[level] = len(df)

# Create FAISS vectorstore
vectorstore = FAISS.from_documents(all_documents, embeddings)

print(f" Vectorstore created with {len(all_documents)} total documents!")
print(f" Distribution across levels: {level_distribution}")

 Building unified FAISS vectorstore...
 Adding Level 0 to vectorstore...
 Adding Level 1 to vectorstore...
 Adding Level 2 to vectorstore...
 Adding Level 3 to vectorstore...
 Vectorstore created with 253 total documents!
 Distribution across levels: {0: 239, 1: 8, 2: 4, 3: 2}


In [83]:
# Create contextual compression retriever
print(" Creating contextual compression retriever...")

# Base retriever
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 8})

# Compression prompt to extract only relevant information
compression_prompt = ChatPromptTemplate.from_template(
    "Given the context and question, extract only the information that's directly relevant to answering the question:\n\n"
    "Context: {context}\n"
    "Question: {question}\n\n"
    "Relevant Information:"
)

# Create the extractor
extractor = LLMChainExtractor.from_llm(llm, prompt=compression_prompt)

# Create compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=extractor,
    base_retriever=base_retriever
)

print(" Smart retriever created!")

 Creating contextual compression retriever...
 Smart retriever created!


In [84]:
def test_raptor_query(query, retriever, show_sources=True):
    """Test RAPTOR with a query and show detailed results."""
    print(f"\n Testing query: '{query}'")
    print("=" * 60)

    # Retrieve documents
    retrieved_docs = retriever.invoke(query)

    if show_sources:
        print(f"📄 Retrieved {len(retrieved_docs)} relevant documents:")
        level_counts = {}

        for i, doc in enumerate(retrieved_docs):
            level = doc.metadata.get('level', 'unknown')
            level_counts[level] = level_counts.get(level, 0) + 1

            print(f"\n  {i+1}. Level {level} | {doc.page_content[:120]}...")

        print(f"\n Level distribution: {level_counts}")

    # Generate answer
    if retrieved_docs:
        context = "\n\n".join([doc.page_content for doc in retrieved_docs])

        answer_prompt = ChatPromptTemplate.from_template(
            "Based on the following context, provide a comprehensive answer:\n\n"
            "Context: {context}\n\n"
            "Question: {question}\n\n"
            "Answer:"
        )

        chain = answer_prompt | llm
        result = chain.invoke({"context": context, "question": query})
        answer = result.content if hasattr(result, 'content') else str(result)

        print(f"\n RAPTOR Answer:")
        print("-" * 40)
        print(answer)

        return {
            "answer": answer,
            "sources": len(retrieved_docs),
            "levels_used": list(level_counts.keys()) if show_sources else []
        }
    else:
        print(" No relevant documents found.")
        return None

# Test with multiple queries
test_queries = [
    "Which factors most significantly influenced fund flows of fixed-income mutual funds during the pre-crisis period (2017–2018)? How did short-term interest rates affect these flows?",
    "How did investor risk aversion manifest in fund flows during the 2019–2022 crisis period? Did fund size or NAV significantly impact inflows?",
    "Which fund-specific characteristics, such as monthly return or NAV, drove investor decisions in the post-crisis period (2023–2024)?",
    "How did 3-month Treasury Bill yields and 10-year government bond yields differently affect fund flows across the three economic periods?",
    "What role did the ASPI index play in shaping investor behavior for fixed-income mutual funds during crises compared to stable periods?",
    "How did investor preferences shift between safety, liquidity, and performance across pre-crisis, crisis, and post-crisis periods?",
    "Which macroeconomic variables (interest rates, bond yields, stock index) are the strongest predictors of fund flows in emerging markets like Sri Lanka?",
    "How does the flow-performance relationship of fixed-income mutual funds differ between crisis and non-crisis periods?",
    "What investment strategies should fund managers adopt to optimize inflows during volatile versus stable economic conditions?",
    "How can monetary policies and liquidity management during crises stabilize fund flows and improve market confidence for fixed-income mutual funds?"
]

results = []
for query in test_queries:
    result = test_raptor_query(query, compression_retriever)
    if result:
        results.append(result)
        print(f" Used {result['sources']} sources from levels {result['levels_used']}")


 Testing query: 'Which factors most significantly influenced fund flows of fixed-income mutual funds during the pre-crisis period (2017–2018)? How did short-term interest rates affect these flows?'
📄 Retrieved 8 relevant documents:

  1. Level 1 | During the pre-crisis period, fund flows of fixed-income mutual funds were largely influenced by short-term interest rat...

  2. Level 1 | The study investigates the determinants of fund flows across pre-crisis, crisis, and post-crisis periods, with a specifi...

  3. Level 1 | During the pre-crisis period (2017-2018), investors prioritized safety with limited sensitivity to returns, reflecting r...

  4. Level 0 | - Factors influencing fund flows of fixed-income mutual funds during the pre-crisis period (2017–2018).
- Impact of shor...

  5. Level 1 | - U.S. income funds outperformed European counterparts, providing a safer alternative to volatile equity markets.
- The ...

  6. Level 0 | The significant determinant factors of fund flow fo